In [0]:
%tensorflow_version 1.x
!git clone https://github.com/tensorflow/cleverhans.git
!pip install cleverhans/
!pip install adversarial-robustness-toolbox

TensorFlow 1.x selected.
Cloning into 'cleverhans'...
remote: Enumerating objects: 13501, done.
remote: Total 13501 (delta 0), reused 0 (delta 0), pack-reused 13501
Receiving objects: 100% (13501/13501), 8.40 MiB | 6.30 MiB/s, done.
Resolving deltas: 100% (9494/9494), done.
Processing ./cleverhans
     |████████████████████████████████| 163kB 2.7MB/s 
     |████████████████████████████████| 51kB 5.2MB/s 
  Created wheel for cleverhans: filename=cleverhans-3.0.1-cp36-none-any.whl size=253453 sha256=cf36ae6f3cdc31ed71f0f3439d4b589b482eeaacf2357bbf2a76e88fb43bb26d
  Stored in directory: /tmp/pip-ephem-wheel-cache-bg_4kw9b/wheels/d1/6b/1d/5cf7b3ca4c0cfc7f845628b8ed46366ab5f4f56b5483e9db7f
Successfully built cleverhans
     |████████████████████████████████| 491kB 2.8MB/s 
     |████████████████████████████████| 7.1MB 12.1MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [0]:
  
from __future__ import absolute_import, division, print_function, unicode_literals
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout
import numpy as np
import matplotlib.pyplot as plt
from art.attacks import FastGradientMethod
from art.classifiers import KerasClassifier
from art.utils import load_dataset
import tensorflow as tf
import tensorflow_hub as hub
tf.compat.v1.disable_eager_execution()

# Load the CIFAR 10 dataset
(x_train, y_train), (x_test, y_test), min_, max_ = load_dataset(str("cifar10")) # Original Dataset
print("x_train shape: " + str(x_train.shape) + "\n" + "x_train size: " + str(x_train.size) + "\n" +
      "y_train shape: " + str(y_train.shape) + "\n" + "y_train size: " + str(y_train.size) + "\n" +
      "x_test shape: " + str(x_test.shape) + "\n" + "x_test size: " + str(x_test.size) + "\n" +
      "y_test shape: " + str(y_test.shape) + "\n" + "y_test size: " + str(y_test.size) + "\n")

#  Load the victim model
classifier_url ="https://tfhub.dev/deepmind/ganeval-cifar10-convnet/1" #@param {type:"string"} # model is downloaded from this site
IMAGE_SHAPE = (32, 32) # the image shape is needed so that the model knows the input-shape and since we are working with the CIFAR-10 all the images are 32 x 32 color images
victim_classifier = KerasClassifier(model=tf.keras.Sequential([hub.KerasLayer(classifier_url, input_shape=IMAGE_SHAPE+(3,))]), clip_values=(min_, max_)) # this bascially creates a keras wrapper around the downloaded model so that we can use it with keras functions.


#Evaluate the victim model on the benign dataset
predictions = victim_classifier.predict(x_test) # giving the classifier the x_test of the CIFAR-10 dataset.
accuracy_benign = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test) # calculates the accuracy of the predictions
print("Accuracy on benign test examples for victim classifier: {}%\n".format(accuracy_benign * 100))


Using TensorFlow backend.


x_train shape: (50000, 32, 32, 3)
x_train size: 153600000
y_train shape: (50000, 10)
y_train size: 500000
x_test shape: (10000, 32, 32, 3)
x_test size: 30720000
y_test shape: (10000, 10)
y_test size: 100000

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Accuracy on benign test examples for victim classifier: 94.52000000000001%



In [0]:
# Step 2: Create the model
model = Sequential()
model.add(Conv2D(32, (3, 3), padding="same", input_shape=x_train.shape[1:]))
model.add(Activation("relu"))
model.add(Conv2D(32, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(10))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()


#Create the classifier
thieved_classifier = KerasClassifier(model=model, clip_values=(min_, max_))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)       

In [0]:
from art.attacks.extraction import KnockoffNets
extraction_attack = KnockoffNets(classifier=victim_classifier, batch_size_fit=128, batch_size_query=128, nb_epochs=10, nb_stolen=1000000) 
stolen_classifier = extraction_attack.extract(x=x_train[:2500], thieved_classifier=thieved_classifier)
predictions = stolen_classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

Accuracy on benign test examples: 4.81%


In [0]:
extraction_attack = KnockoffNets(classifier=victim_classifier, batch_size_fit=128, batch_size_query=128, nb_epochs=10, nb_stolen=1000000) 
stolen_classifier = extraction_attack.extract(x=x_train[:5000], thieved_classifier=thieved_classifier)
predictions = stolen_classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

Accuracy on benign test examples: 10.0%


In [0]:
extraction_attack = KnockoffNets(classifier=victim_classifier, batch_size_fit=128, batch_size_query=128, nb_epochs=10, nb_stolen=1000000) 
stolen_classifier = extraction_attack.extract(x=x_train[:15000], thieved_classifier=thieved_classifier)
predictions = stolen_classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

Accuracy on benign test examples: 15.9%


In [0]:
extraction_attack = KnockoffNets(classifier=victim_classifier, batch_size_fit=128, batch_size_query=128, nb_epochs=10, nb_stolen=2500) 
stolen_classifier = extraction_attack.extract(x=x_train[:2500], thieved_classifier=thieved_classifier)
predictions = stolen_classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

Accuracy on benign test examples: 14.42%


In [0]:
extraction_attack = KnockoffNets(classifier=victim_classifier, batch_size_fit=128, batch_size_query=128, nb_epochs=10, nb_stolen=5000) 
stolen_classifier = extraction_attack.extract(x=x_train[:5000], thieved_classifier=thieved_classifier)
predictions = stolen_classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

Accuracy on benign test examples: 12.21%


In [0]:
extraction_attack = KnockoffNets(classifier=victim_classifier, batch_size_fit=128, batch_size_query=128, nb_epochs=10, nb_stolen=15000) 
stolen_classifier = extraction_attack.extract(x=x_train[:15000], thieved_classifier=thieved_classifier)
predictions = stolen_classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

Accuracy on benign test examples: 10.07%
